In [ ]:
# Emotion Detection with TensorFlow, MobileNetV2 and OpenCV

# Install required libraries
!pip install tensorflow opencv-python matplotlib

# Import required packages
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

# Prepare data generators for training and validation
train_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, zoom_range=0.2)
val_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    'data/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_data = val_gen.flow_from_directory(
    'data/validation',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Build the model using MobileNetV2 for transfer learning
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(train_data.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Add callbacks for early stopping and saving the best model
callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ModelCheckpoint('best_emotion_model.h5', save_best_only=True)
]

# Train the model with callbacks
history = model.fit(train_data, validation_data=val_data, epochs=20, callbacks=callbacks)

# Save the trained model
model.save('emotion_model_mobilenet')

# Plot training history
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.title('Model Performance')
plt.legend()
plt.grid(True)
plt.show()

# Load the trained model and class labels for real-time detection
model = tf.keras.models.load_model('emotion_model_mobilenet')
class_names = list(train_data.class_indices.keys())

# Initialize webcam and face detection
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start real-time emotion detection loop
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame.")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            roi_color = frame[y:y+h, x:x+w]
            roi_resized = cv2.resize(roi_color, (224, 224)) / 255.0
            roi_expanded = np.expand_dims(roi_resized, axis=0)
            prediction = model.predict(roi_expanded)
            emotion = class_names[np.argmax(prediction)]
            confidence = np.max(prediction)
            label = f"{emotion} ({confidence*100:.1f}%)"

            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        cv2.imshow('Emotion Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except Exception as e:
    print(f"Error: {e}")

finally:
    cap.release()
    cv2.destroyAllWindows()

# Optional: Predict emotion from a static image file
from tensorflow.keras.preprocessing import image

img = image.load_img('test.jpg', target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

pred = model.predict(img_array)
print("Predicted emotion:", class_names[np.argmax(pred)])
